In [ ]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
import numpy as np

In [ ]:
def minstIID(dataset, num_users):
    images = int(len(dataset)/num_users)
    users_dict, indeces = {}, [i for i in range(len(dataset))]
    for i in range(num_users):
        np.random.seed(i)
        users_dict[i] = set(np.random.choice(indeces, images, replace=False))
        indeces = list(set(indeces) - users_dict[i])
    return users_dict

In [ ]:
def load_dataset(num_users):
    print('Hello and Welcome')
    tranform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])
    train_dataset = datasets.MNIST('./data', train=True, download=True, transform=tranform)
    test_dataset = datasets.MNIST('./data', train=False, download=True, transform=tranform)
    train_group= minstIID(train_dataset,num_users)
    test_group= minstIID(test_dataset,num_users)
    return train_dataset, test_dataset, train_group, test_group

In [ ]:
class FedDataset( Dataset):
    def __init__(self, dataset, indx):
        self.dataset=dataset
        self.indx=[int(i) for i in indx]
    
    def __len__(self):
        return len(self.indx)
    
    def __getitem__(self, item):
        images, label=self.dataset[self.indx[item]]
        return torch.tensor(images).clone().detach(), torch.tensor(label).clone().detach()
    

In [ ]:
def getActualImgs(dataset, indeces, batch_size):
    return DataLoader(FedDataset(dataset ,indeces) , batch_size=batch_size, shuffle=True)

In [ ]:
train_dataset, test_dataset, train_group, test_group=load_dataset(10)

Hello and Welcome


In [ ]:
first_client_batches=getActualImgs(train_dataset, train_group[0], 64)

In [ ]:
for images, labels in first_client_batches:
    print(len(images))
    break

64


<ipython-input-15-8c6ecfb6927a>:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(images).clone().detach(), torch.tensor(label).clone().detach()
